<a href="https://colab.research.google.com/github/MartinAtCE/StockpredictionPublic/blob/master/StockPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q quandl

     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 61kB 25.2MB/s 
     |████████████████████████████████| 2.3MB 14.4MB/s 
     |████████████████████████████████| 102kB 34.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [0]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [4]:
#Get the stock data
df = quandl.get ("WIKI/FB")
# Take a look at the data
print(df.head())

             Open   High    Low  ...  Adj. Low  Adj. Close  Adj. Volume
Date                             ...                                   
2012-05-18  42.05  45.00  38.00  ...     38.00     38.2318  573576400.0
2012-05-21  36.53  36.66  33.00  ...     33.00     34.0300  168192700.0
2012-05-22  32.61  33.59  30.94  ...     30.94     31.0000  101786600.0
2012-05-23  31.37  32.50  31.36  ...     31.36     32.0000   73600000.0
2012-05-24  32.95  33.21  31.77  ...     31.77     33.0300   50237200.0

[5 rows x 12 columns]


In [5]:
# Get the adjustead close
df = df[['Adj. Close']]
# Take a look at reduced data
print(df.head())

            Adj. Close
Date                  
2012-05-18     38.2318
2012-05-21     34.0300
2012-05-22     31.0000
2012-05-23     32.0000
2012-05-24     33.0300


In [6]:
# A variable for predicting 'n' days into the future
forecast_out = 30
# Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
print(df.head())

            Adj. Close  Prediction
Date                              
2012-05-18     38.2318      30.771
2012-05-21     34.0300      31.200
2012-05-22     31.0000      31.470
2012-05-23     32.0000      31.730
2012-05-24     33.0300      32.170


In [7]:
### Create the independent data set (x) ###
# Convert the dataframe to numpy array"
X = np.array(df.drop(['Prediction'], 1))
#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


In [8]:
### Create the dependent data set (Y) ###
# Convert the dataframe to a numpy array (all of the values incl NaN's)
y = np.array(df['Prediction'])
# Get all the data except for the last 'n' rows
y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [0]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [11]:
# Testing model: Score returns the coefficuent of determination R2 of the prediction
# Best score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.982357237678363


In [12]:
# Create and train the linear regression model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
# Testing model: Score returns the coefficuent of determination R2 of the prediction
# Best score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9771742742118349


In [0]:
# https://www.youtube.com/watch?v=EYnC4ACIt2g @ 30:00

# Set x_forecast equal to the last 30 rows of the original data set from Adj Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

In [18]:
# Print the prediction for the next 'n days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print the support vector regressor prediction for the next 'n days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[176.8676221  183.30002184 183.74433203 181.11886275 179.75563831
 181.67425048 182.76483003 187.1069523  188.76301754 185.25902585
 182.08826679 179.6849526  180.3716138  184.18864222 183.56256877
 187.53106657 186.14764622 189.06595631 188.59135224 185.68314012
 188.01576859 187.68253595 188.92458488 176.27184254 171.81864272
 173.0707896  168.52670816 162.97283083 163.64939407 155.70230051]
[177.8349214  181.46927108 181.54342799 176.1868388  175.19010097
 177.36625789 180.57863622 179.28392096 178.40204269 180.23661949
 178.59025246 175.15291088 175.48977679 181.16765147 181.581682
 178.52362469 180.21531581 179.2125276  178.08817579 180.21338355
 177.88776394 178.27484863 178.79743321 178.63315182 171.1268751
 173.11007755 172.16725395 167.55769938 166.26930324 157.36471486]
